In [ ]:
!pip install --upgrade --pre pythainlp
!pip install pyLDAvis

In [ ]:
import pandas as pd
import pythainlp
import gensim
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
df=pd.read_csv('mont_nomsod_wongnai.csv')

In [ ]:
df.tail()

,ReviewID,Review
15,16,สั่งไลน์แมนมาส่ง เพราะอยากกินปังปิ้งช่วงบ่าย เ...
16,17,นั่งทานในร้าน หาที่นั่งยากมากค่ะ คนเต็มตลอด ที...
17,18,อร่อย หอม นุ่มมมม\nสั่งสังขยาแล้วก็สังขยาใบเตย...
18,19,ให้ห้าดาวเนื่องจากส่วนตัวชอบมาก ขนมปังปิ้งและน...
19,20,ทางร้านปรับปรุงโซนจำหน่ายเครื่องดื่ม ทำขนมปัง ...


In [ ]:
# Tokenize Words
stopwords = list(pythainlp.corpus.thai_stopwords())
removed_words = ['',' ','  ','\n', '/','-','ค่ะ','คะ','ร้าน', 'มนต์','นะคะ', 'ครับ',
                 'ๆ','('')','(',')','(' ,')','(' , ')','(', ')','มาก',
                 'บริการ','ดีมาก','อย่างมาก','เลย','แนะนำ','ไม่มีที่ติ','เหมาะกับ',
                 'มมม','ความ','แยก','ไร้','หาย','ติ','เรื่อง','ให้การ','เข้าหา',
                 'ดี','น่ารัก','ยอดเยี่ยม','สุดยอด','ดีงาม','อัธ', 'เยี่ยม', 'ขนาด',
                 'สาขา', 'เสา', 'ชิงช้า', '2', '5', 'EQ', 'ตอน', 'ตอนนี้', 'ศาลา', 'ว่าการ',
                 'แห', '15', '20', '25', '3', '35', '4', '555', '30', '60', 'จ่ะ',
                 'ส่วนตัว', 'บาท', 'อก', 'า', 'ชอก', 'ต', 'กก', 'มีประวัติ', 'รึเปล่า', 'บดี', 'มดี',
                 'ฃ', '!”', '“', '.', '10', '://', '=', '?','com', 'ct', 'https',
                 'montnomsod-dinsor', 'ref', 'restaurants', 'reviews', 'wongnai', 'www',
                 'ผม', 'สิริ','อ่าน', 'เติม', 'กา', 'น', 'ๆๆๆ']
screening_words = stopwords + removed_words

def tokenize_with_space(sentence):
    merged = ''
    words = pythainlp.word_tokenize(str(sentence), engine='newmm')
    for word in words:
        if word not in screening_words:
            merged = merged + ',' + word
    return merged[1:]

In [ ]:
df['Review_tokenized'] = df['Review'].apply(lambda x: tokenize_with_space(x))

In [ ]:
df.tail()

,ReviewID,Review,Review_tokenized,topics,score
15,16,สั่งไลน์แมนมาส่ง เพราะอยากกินปังปิ้งช่วงบ่าย เ...,"สั่ง,ไลน์,แมน,กิน,ปัง,ปิ้ง,บ่าย,รอ,บ่าย,สอง,สั...",1,0.996052
16,17,นั่งทานในร้าน หาที่นั่งยากมากค่ะ คนเต็มตลอด ที...,"นั่ง,ทาน,หา,ที่นั่ง,คน,เต็ม,ที่นั่ง,แคบ,ถนัดตา...",0,0.998556
17,18,อร่อย หอม นุ่มมมม\nสั่งสังขยาแล้วก็สังขยาใบเตย...,"อร่อย,หอม,นุ่ม,สั่ง,สังขยา,แล้วก็,สังขยา,ใบ,เต...",2,0.997690
18,19,ให้ห้าดาวเนื่องจากส่วนตัวชอบมาก ขนมปังปิ้งและน...,"ห้า,ดาว,ชอบ,ขนมปัง,ปิ้ง,นมสด,เมนู,โปรด,ขนมปัง,...",0,0.997818
19,20,ทางร้านปรับปรุงโซนจำหน่ายเครื่องดื่ม ทำขนมปัง ...,"ปรับปรุง,โซน,จำหน่าย,เครื่องดื่ม,ทำ,ขนมปัง,กั้...",2,0.992542


In [ ]:
# Create Dictionary
documents = df['Review_tokenized'].to_list()
texts = [[text for text in doc.split(',')] for doc in documents]
dictionary = gensim.corpora.Dictionary(texts)

In [ ]:
print(dictionary.token2id.keys())

dict_keys(['กรุงเทพ', 'กลับบ้าน', 'กี่', 'ขนมปัง', 'คน', 'ความรู้สึก', 'คุ้นเคย', 'ชอบ', 'ซื้อ', 'ดู', 'ตรงข้าม', 'ตัวเรา', 'ติดแอร์', 'ทาน', 'ทำ', 'ที่จะ', 'ที่อื่น', 'ทุ่ม', 'ธรรมดา', 'นม', 'นมสด', 'นิสัยเสีย', 'บรรยากาศ', 'บ่าย', 'ผสม', 'ผิดที่', 'พัฒนา', 'รสชาติ', 'รอ', 'ระบบ', 'รำลึก', 'ลงตัว', 'ล่ะ', 'ว่าไม่ได้', 'สัก', 'หลายอย่าง', 'ห้องแถว', 'อันนี้', 'เจอ', 'เด็ก', 'เป็นหนึ่ง', 'เมนู', 'เลือก', 'เวลา', 'แน่น', 'แย่', 'โมง', 'โอเค', 'กร้าน', 'กิน', 'ขยาย', 'ความอ้วน', 'คิว', 'ชื่น', 'ดั้งเดิม', 'ตรงนี้', 'ที่นั่ง', 'ที่นี่', 'นอ', 'นิดนึง', 'นึ่ง', 'น้ำ', 'น้ำตาล', 'น้ำแข็ง', 'ปกติ', 'ปิ้ง', 'ยังมี', 'รู้สึก', 'ลอง', 'ล้นหลาม', 'สังขยา', 'สั่ง', 'สเตชั่น', 'หน้า', 'หลัก', 'หวาน', 'หอม', 'หักคะแนน', 'อร่อย', 'เก้าอี้ดนตรี', 'เชียว', 'เต็ม', 'เนย', 'เย็น', 'เสมอ', 'แก้ว', 'แดง', 'แบ่ง', 'แย่งชิง', 'แห้ง', 'ใจดี', 'ไข่', 'ไปนอก', 'กรอบ', 'กลิ่น', 'ขาดไม่ได้', 'ความนุ่ม', 'ความหวาน', 'คู่', 'ชา', 'ชุด', 'ตราด', 'ตำหรับ', 'นุ่ม', 'นุ่มนวล', 'มะนาว', 'ยืน', 'ร้อน', 'ล้น', 'สดชื่น', '

In [ ]:
gensim_corpus = [dictionary.doc2bow(text, allow_update=True) for text in texts]
word_frequencies = [[(dictionary[id], frequence) for id, frequence in couple] for couple in gensim_corpus]

In [ ]:
#Topic Modeling
num_topics = 3
chunksize = 4000 # size of the doc looked at every pass
passes = 30 # number of passes through documents
iterations = 50
eval_every =1 # Don't evaluate model perplexity, takes too much time

#make a index to word dictionary
temp = dictionary[0] #This is only to load the dictionary
id2word = dictionary.id2token

%time model = gensim.models.LdaModel(corpus=gensim_corpus, id2word=id2word, chunksize=chunksize, \
                                    alpha='auto', eta='auto', \
                                    iterations=iterations, num_topics=num_topics, \
                                    passes=passes, eval_every=eval_every)

CPU times: user 655 ms, sys: 0 ns, total: 655 ms
Wall time: 688 ms


In [ ]:
pyLDAvis.gensim.prepare(model, gensim_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.077436  0.055202       1        1  41.572385
1     -0.036804 -0.070051       2        1  31.292789
0      0.114240  0.014850       3        1  27.134826, topic_info=            Term       Freq      Total Category  logprob  loglift
75          หวาน  14.000000  14.000000  Default  30.0000  30.0000
65          ปิ้ง  14.000000  14.000000  Default  29.0000  29.0000
134         โต๊ะ   4.000000   4.000000  Default  28.0000  28.0000
22      บรรยากาศ   4.000000   4.000000  Default  27.0000  27.0000
144          รู้   4.000000   4.000000  Default  26.0000  26.0000
..           ...        ...        ...      ...      ...      ...
41          เมนู   1.609848  12.769598   Topic3  -4.9834  -0.7666
133  เครื่องดื่ม   1.609828   7.250344   Topic3  -4.9834  -0.2006
117       เพื่อน   1.609789   2.880351   Topic3  -4.9834   0.7225
247     ที่จอดรถ   1.609778   2.880354   Topic3  -4.9834   0.7225
42         เลือก   1.609772   5.199700   Topic3  -4.9834   0.1318

[190 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
179       2  0.923275        MBK
135       3  0.949071  Signature
93        1  0.506832       กรอบ
93        2  0.168944       กรอบ
93        3  0.337888       กรอบ
...     ...       ...        ...
134       3  0.816926       โต๊ะ
123       1  0.879643       โปรด
91        1  0.303494        ไข่
91        2  0.758735        ไข่
281       2  0.923275       ไลน์

[201 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [ ]:
model.show_topic(1)

[('ขนมปัง', 0.046016872),
 ('สังขยา', 0.032758858),
 ('หวาน', 0.030084765),
 ('สั่ง', 0.022127718),
 ('ปิ้ง', 0.022121107),
 ('นมสด', 0.016811958),
 ('ไข่', 0.01681177),
 ('ชอบ', 0.014165448),
 ('กิน', 0.014157505),
 ('เครื่องดื่ม', 0.014157505)]

In [ ]:
df['topics'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][0])
df['score'] = df['Review_tokenized'].apply(lambda x: model.get_document_topics(dictionary.doc2bow(x.split(',')))[0][1])

In [ ]:
df.tail()

,ReviewID,Review,Review_tokenized,topics,score
15,16,สั่งไลน์แมนมาส่ง เพราะอยากกินปังปิ้งช่วงบ่าย เ...,"สั่ง,ไลน์,แมน,กิน,ปัง,ปิ้ง,บ่าย,รอ,บ่าย,สอง,สั...",1,0.998314
16,17,นั่งทานในร้าน หาที่นั่งยากมากค่ะ คนเต็มตลอด ที...,"นั่ง,ทาน,หา,ที่นั่ง,คน,เต็ม,ที่นั่ง,แคบ,ถนัดตา...",2,0.998833
17,18,อร่อย หอม นุ่มมมม\nสั่งสังขยาแล้วก็สังขยาใบเตย...,"อร่อย,หอม,นุ่ม,สั่ง,สังขยา,แล้วก็,สังขยา,ใบ,เต...",0,0.998845
18,19,ให้ห้าดาวเนื่องจากส่วนตัวชอบมาก ขนมปังปิ้งและน...,"ห้า,ดาว,ชอบ,ขนมปัง,ปิ้ง,นมสด,เมนู,โปรด,ขนมปัง,...",2,0.998316
19,20,ทางร้านปรับปรุงโซนจำหน่ายเครื่องดื่ม ทำขนมปัง ...,"ปรับปรุง,โซน,จำหน่าย,เครื่องดื่ม,ทำ,ขนมปัง,กั้...",1,0.996428
